In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from isaacgym import gymtorch
from isaacgym import gymapi
from utils.rlgames_utils import get_rlgames_env_creator
import train
from rl_games.torch_runner import Runner

from omegaconf import OmegaConf
from hydra import compose, initialize
import yaml
from utils import reformat
from isaacgymenvs.utils.rlgames_utils import RLGPUAlgoObserver
import torch
import numpy as np
import time
import os
os.environ['DISPLAY'] = ':0'

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.8.1
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


2022-06-30 15:37:10,229 - INFO - logger - logger initialized


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


In [3]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task.env.numEnvs=2", "task.env.observationType=full", 
                                                   "task.env.objectType=dispenser", "test=true",
                                                   "sim_device=cpu"])

env_loader = get_rlgames_env_creator(
    reformat.omegaconf_to_dict(cfg.task), 'AllegroHandGrasp', cfg.sim_device,
    cfg.rl_device, cfg.graphics_device_id, cfg.headless)

env = env_loader()

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Obs type: full
GPU Pipeline can only be used with GPU simulation. Forcing CPU Pipeline.
Not connected to PVD
Physics Engine: PhysX
Physics Device: cpu
GPU Pipeline: disabled
Num dofs:  16
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Num dofs:  20


env.shadow_hand_dof_pos[0].detach().cpu() - old

old = env.shadow_hand_dof_pos[0].detach().cpu().numpy()

In [4]:
def get_action(t):
    hand_pos = np.zeros((env.num_envs, 3))
    hand_pos[:, 2] = np.sin(t) * 0.25 + 0.25
    actions = np.stack([env.action_space.sample() for _ in range(env.num_envs)])
    action = np.hstack((hand_pos, actions))
    return torch.tensor(action, device=env.device).float()

## Get camera transform

Saving camera transform

```python
transform = env.gym.get_viewer_camera_transform(env.viewer, env.envs[0])
pos = transform.p
print("position", pos.x, pos.y, pos.z)

rot = transform.r
print("rotation", rot.x, rot.y, rot.z, rot.w)

import pickle
pickle.dump(transform, open('allegro_viewer.pkl', 'wb'))
```

https://arxiv.org/pdf/2002.12361.pdf```python
import pickle

transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
env.gym.set_camera_transform(env.viewer, env.envs[0], transform)
```

In [5]:
obs = env.reset()
action = get_action(0)
obs, r, done, info = env.step(action)

In [ ]:
def move_to_object(t):
    obj_pos = env.object_pos
    
    hand_pos = np.zeros((env.num_envs, 3))
    hand_pos[:, 2] = np.sin(t) * 0.25 + 0.25
    actions = np.stack([env.action_space.sample() for _ in range(env.num_envs)])
    action = np.hstack((hand_pos, actions))
    return torch.tensor(action, device=env.device).float()

In [6]:
# obs = env.reset()
# action = get_action()
# obs, r, done, info = env.step(action)
t = 0

# moves hand up and down
while True:
    t += 1
    action = get_action(t)
    obs, r, done, info = env.step(action)
    env_ids = done.nonzero(as_tuple=False).squeeze(-1)
    goal_env_ids = env.reset_goal_buf.nonzero(as_tuple=False).squeeze(-1)
    time.sleep(0.01)
    if done.any():
        env.reset_idx(env_ids, goal_env_ids)

KeyboardInterrupt: 